In [26]:
# The code was removed by Watson Studio for sharing.

# Web scrapping the boroughs and neighbourhoods in Toronto

### 1. Importing necessary libraries

For this assignment, we shall use pandas library to read the HTML table from the wiki page. Also we would require lxml parser to support it

In [27]:
import requests
import lxml.html as lh
import pandas as pd

print("Libraries imported")

Libraries imported


### 2. Extract data from source

We shall read the html source with requests and extract the table data from it

In [28]:
#Source URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a page handle to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().rstrip('\n') 
    col.append((name,[]))
    

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    cburough = None
    neighborhood = None
    postcode = None
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().rstrip('\n') 
        #Assign first column to postcode
        if i==0:
            postcode = data
        #Assign second column to burough
        if i==1:
            cburough = data
            #If burough is having value 'Not assigned', ignore this row
            if data == 'Not assigned':
                break
        #Assign the third column to neighborhood
        if i==2:
            neighborhood = data;
            #If neighborhood is having the value 'Not assigned', assign it with borough's name
            if data == 'Not assigned':
                neighborhood = cburough
        #Increment i for the next column
        i+=1
    #Append the data to the empty list of the all three columns only when all column has values
    if postcode != None and cburough != None and neighborhood != None:
        col[0][1].append(postcode)
        col[1][1].append(cburough)
        col[2][1].append(neighborhood)

#Convert the list to pandas dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
#Group the neighborhoods with same postcode to be put together as comma separated
df = df.groupby(["Postcode","Borough"])['Neighbourhood'].apply(','.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
# Save dataframe as csv file to storage for later processing
project.save_data(data=df.to_csv(index=False),file_name='toronto_dataframe.csv',overwrite=True)
#Print the dataframe shape
df.shape

(103, 3)